In [11]:
from data import data_from_dict
import requests
import json

In [12]:
def stripNone(data):
  if isinstance(data, dict):
      return {k:stripNone(v) for k, v in data.items() if k is not None and v is not None}
  elif isinstance(data, list):
      return [stripNone(item) for item in data if item is not None]
  elif isinstance(data, tuple):
      return tuple(stripNone(item) for item in data if item is not None)
  elif isinstance(data, set):
      return {stripNone(item) for item in data if item is not None}
  else:
      return data

In [31]:
URI = 'https://api.cian.ru/search-offers/v2/search-offers-desktop/'
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
}
data = {
    "jsonQuery": {
        "region": {
            "type": "terms",
            "value": [4704, 4707]
        },
        "publish_period": {
            "type": "terms",
            "value": [60]
        },
        "_type": "commercialsale",
#         'category': {
#             'type': 'terms',
#             'value': [
#                 'commercialLandSale',
#                 'garageSale',
#                 'businessSale',
#                 'buildingSale',
#                 'officeSale',
#                 'freeAppointmentObjectSale',
#                 'shoppingAreaSale',
#                 'industrySale'
#             ]},
        "office_type": {
            "type": "terms",
            "value": [1,  # Офис
                      2,  # Торговая площадь
                      4,  # ПСН
                      5,  # Общепит
                      10,  # Здание
#                       11,  # Бытовые услуги
#                       3,  # Склад
#                       6,  # Гараж
#                       7,  # Производство
#                       8,  # Автосервис
#                       9,  # Готовый бизнес
#                       12  # Коммерческая земля
                      ]
        },
        "engine_version": {
            "type": "term",
            "value": 2
        },
        "page": {
            "type": "term",
            "value": 1
        }
    }
}
response = requests.post(URI, headers=headers, data=json.dumps(data))

In [32]:
result = data_from_dict(response.json()).data
# result = response.json()['data']
result.aggregated_count

793

In [23]:
pages = result.aggregated_count // len(result.offers_serialized) + 2
pages

43

In [24]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.cian
collection = db.offers
collection.create_index([('id', pymongo.ASCENDING)], unique=True)

'id_1'

In [25]:
for offer in result.offers_serialized:
    offer_compact = stripNone(offer.to_dict())
    collection.find_one_and_replace(
        {"id": offer.id}, offer_compact, upsert=True)

In [13]:
with open("output.txt", "a+") as text_file:
    print(response.text, file=text_file)

In [ ]:
for x in range (2,pages):
    data['page']['value'] = x
    response = requests.post(
        'https://www.cian.ru/cian-api/mobile-site/v2/search-offers/', headers=headers, data=json.dumps(data))
    with open("output.txt", "a+") as text_file:
        print(response.text, file=text_file)
    result = data_from_dict(response.json())
    print(result.data.offers_serialized)